In [35]:
# Code in file autograd/two_layer_net_autograd.py
import torch

device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Create random Tensors for weights; setting requires_grad=True means that we
# want to compute gradients for these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
  # Forward pass: compute predicted y using operations on Tensors. Since w1 and
  # w2 have requires_grad=True, operations involving these Tensors will cause
  # PyTorch to build a computational graph, allowing automatic computation of
  # gradients. Since we are no longer implementing the backward pass by hand we
  # don't need to keep references to intermediate values.
  y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
  # Compute and print loss. Loss is a Tensor of shape (), and loss.item()
  # is a Python number giving its value.
  loss = (y_pred - y).pow(2).sum()
#   print(t, loss.item())

  # Use autograd to compute the backward pass. This call will compute the
  # gradient of loss with respect to all Tensors with requires_grad=True.
  # After this call w1.grad and w2.grad will be Tensors holding the gradient
  # of the loss with respect to w1 and w2 respectively.
  loss.backward()

  # Update weights using gradient descent. For this step we just want to mutate
  # the values of w1 and w2 in-place; we don't want to build up a computational
  # graph for the update steps, so we use the torch.no_grad() context manager
  # to prevent PyTorch from building a computational graph for the updates
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    # Manually zero the gradients after running the backward pass
    w1.grad.zero_()
    w2.grad.zero_()

In [54]:
# Code in file tensor/two_layer_net_tensor.py
import torch

device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 64, 1000, 500, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Randomly initialize weights
w1 = torch.randn(D_in, H1, device=device)/10
w2 = torch.randn(H1, H2, device=device)/10
w3 = torch.randn(H2, D_out, device=device)/10

w4 = w1.clone()
w5 = w2.clone()
w6 = w3.clone()

learning_rate = 1e-6
for t in range(50):
    # Forward pass: compute predicted y
    h1 = x.mm(w1)
    h1_relu = h1.clamp(min=0)
    h2 = h1_relu.mm(w2)
    h2_relu = h2.clamp(min=0)
    y_pred = h2_relu.mm(w3)

    # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
    # of shape (); we can get its value as a Python number with loss.item().
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w3 = h2_relu.t().mm(grad_y_pred)
    grad_h2_relu = grad_y_pred.mm(w3.t())

    grad_h2 = grad_h2_relu.clone()
    grad_h2[h2 < 0] = 0

    grad_w2 = h1_relu.t().mm(grad_h2)
    grad_h1_relu = grad_h2.mm(w2.t())
    grad_h1 = grad_h1_relu.clone()
    grad_h1[h1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h1)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    w3 -= learning_rate * grad_w3
    
    
learning_rate = 1e-6
print('**********************')
for t in range(50):
    # Forward pass: compute predicted y
    h1 = x.mm(w4)
    h1_relu = h1.clamp(min=0)
    h2 = h1_relu.mm(w5)
    h2_relu = h2.clamp(min=0)
    y_pred = h2_relu.mm(w6)

    # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
    # of shape (); we can get its value as a Python number with loss.item().
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w3 = h2_relu.t().mm(grad_y_pred)
    grad_h2_relu = grad_y_pred.mm(w6.t()) * ((torch.sigmoid(h2)-0.5)/0.5)

    grad_h2 = grad_h2_relu.clone()
    grad_h2[h2 < 0] = 0

    grad_w2 = h1_relu.t().mm(grad_h2)
    grad_h1_relu = grad_h2.mm(w5.t()) * ((torch.sigmoid(h1)-0.5)/0.5)
    grad_h1 = grad_h1_relu.clone()
    grad_h1[h1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h1)

    # Update weights using gradient descent
    w4 -= learning_rate * grad_w1
    w5 -= learning_rate * grad_w2
    w6 -= learning_rate * grad_w3    
    
# # Randomly initialize weights
# w3 = torch.randn(D_in, H, device=device)
# print(w3.shape)
# w4 = torch.randn(H, D_out, device=device)

# print('***************************')
# learning_rate = 1e-6
# for t in range(100):
#     # Forward pass: compute predicted y
#     h = x.mm(w3)
#     h_relu = h.clamp(min=0)
# #     print(h_relu.shape)
#     y_pred = (h_relu.mm(w4))
    
#     # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
#     # of shape (); we can get its value as a Python number with loss.item().
#     loss = (y_pred - y).pow(2).sum()
#     print(t, loss.item())

#     # Backprop to compute gradients of w1 and w2 with respect to loss
#     grad_y_pred = 2.0 * (y_pred - y)
# #     print(grad_y_pred.shape)
#     grad_w4 = h_relu.t().mm(grad_y_pred)
# #     print(grad_w4.shape)
#     grad_h_relu = grad_y_pred.mm(w4.t()) * ((torch.sigmoid(h_relu) - 0.5)/0.5)
# #     print(grad_h_relu.shape)
#     grad_h = grad_h_relu.clone()
#     grad_h[h < 0] = 0
#     grad_w3 = x.t().mm(grad_h)

#     # Update weights using gradient descent
#     w3 -= learning_rate * grad_w3
#     w4 -= learning_rate * grad_w4    
# #     adfs

0 9318.6474609375
1 8259.5078125
2 7439.4580078125
3 6790.89697265625
4 6273.5458984375
5 5854.9814453125
6 5510.1240234375
7 5222.58935546875
8 4980.73779296875
9 4774.38818359375
10 4596.90087890625
11 4442.1513671875
12 4305.943359375
13 4185.001953125
14 4076.435302734375
15 3978.0517578125
16 3888.22509765625
17 3805.607666015625
18 3728.853271484375
19 3657.208984375
20 3590.08203125
21 3526.553955078125
22 3466.2119140625
23 3408.604248046875
24 3353.420654296875
25 3300.495849609375
26 3249.51123046875
27 3200.123779296875
28 3152.37158203125
29 3106.15869140625
30 3061.234619140625
31 3017.554443359375
32 2975.029296875
33 2933.555419921875
34 2893.18212890625
35 2853.8427734375
36 2815.459228515625
37 2777.9833984375
38 2741.32080078125
39 2705.40087890625
40 2670.201904296875
41 2635.651123046875
42 2601.817138671875
43 2568.68310546875
44 2536.198486328125
45 2504.3642578125
46 2473.161376953125
47 2442.521484375
48 2412.3984375
49 2382.8134765625
**********************
0 9

In [56]:
# Code in file autograd/two_layer_net_custom_function.py
import torch

class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  @staticmethod
  def forward(ctx, x):
    """
    In the forward pass we receive a context object and a Tensor containing the
    input; we must return a Tensor containing the output, and we can use the
    context object to cache objects for use in the backward pass.
    """
    ctx.save_for_backward(x)
    return x.clamp(min=0)

  @staticmethod
  def backward(ctx, grad_output):
    """
    In the backward pass we receive the context object and a Tensor containing
    the gradient of the loss with respect to the output produced during the
    forward pass. We can retrieve cached data from the context object, and must
    compute and return the gradient of the loss with respect to the input to the
    forward function.
    """
    x, = ctx.saved_tensors
    grad_x = grad_output.clone()
    grad_x[x < 0] = 0
    return grad_x


device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and output
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(50):
  # Forward pass: compute predicted y using operations on Tensors; we call our
  # custom ReLU implementation using the MyReLU.apply function
  y_pred = MyReLU.apply(x.mm(w1)).mm(w2)
 
  # Compute and print loss
  loss = (y_pred - y).pow(2).sum()
  print(t, loss.item())

  # Use autograd to compute the backward pass.
  loss.backward()

  with torch.no_grad():
    # Update weights using gradient descent
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    # Manually zero the gradients after running the backward pass
    w1.grad.zero_()
    w2.grad.zero_()

0 48693164.0
1 58294032.0
2 63886544.0
3 48188224.0
4 22351706.0
5 7506676.0
6 3133998.5
7 1974178.875
8 1519780.625
9 1243464.75
10 1038837.875
11 877102.75
12 746366.625
13 639411.625
14 551005.8125
15 477417.375
16 415716.0625
17 363589.9375
18 319273.3125
19 281406.5
20 248910.34375
21 220886.265625
22 196595.203125
23 175448.265625
24 156995.71875
25 140811.609375
26 126597.90625
27 114061.0390625
28 102966.1640625
29 93120.421875
30 84359.0
31 76548.5625
32 69569.4921875
33 63321.18359375
34 57718.796875
35 52688.89453125
36 48160.58984375
37 44078.359375
38 40391.92578125
39 37055.65625
40 34031.0
41 31285.09765625
42 28788.962890625
43 26515.794921875
44 24443.375
45 22552.43359375
46 20824.763671875
47 19245.14453125
48 17800.16796875
49 16475.767578125
